In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
data=pd.read_csv('/kaggle/input/insurance/insurance.csv')

In [ ]:
data["bmi"]=data["bmi"].astype(int)
data["charges"]=data["charges"].astype(int)

In [ ]:
x=data.iloc[:, :-1].values
y=data.iloc[:,-1].values

In [ ]:
x

In [ ]:
y

### Encode Categorical Data

In [ ]:
## Gender
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
x[:,1]=le.fit_transform(x[:,1])
x[:,4]=le.fit_transform(x[:,4])
x

In [ ]:
#region
np.set_printoptions(threshold=np.sys.maxsize)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [5])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [ ]:
x[:3]

## Test Train Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2, random_state=43)

## Training the Multiple Linear Regression model on the Training set

In [ ]:
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(x_train, y_train)

## ## Predicting the Test set results using the class


In [ ]:
y_pred=regressor.predict(x_test) #### Vector of PREDICTED cost (x_Test)

In [ ]:
np.set_printoptions(precision=2)

In [ ]:
print(np.concatenate((y_pred.reshape(len(y_pred), 1),y_test.reshape(len(y_test),1)),1))


## 1 column - predicted cost, 2 column - real cost
It seems that our model brought closer results to the real charges. Let me know if it makes sense

## Backward Elimination - Manual Elimination

In [ ]:
x=x[:,1:] ## to avoid dummy trap 
len(x)

In [ ]:
np.set_printoptions(threshold=np.sys.maxsize)

In [ ]:
import statsmodels.api as sm
## Create the b0
x=np.append(arr=np.ones((len(x),1)).astype(int),values=x,axis=1)
x[:10]

In [ ]:
x_opt=x[:,[0,1,2,3,4,5,6,7,8]]
x_opt=x_opt.astype(np.float64)

In [ ]:
#Step 2: Fit the full model  with all possible predictors
regressor_OLS=sm.OLS(endog=y,exog=x_opt).fit()
# endog is dependent variable, exog - regressor

Ordinary least-squares (OLS) models assume that the analysis is fitting a model of a relationship between one or more explanatory variables and a continuous or at least interval outcome variable that minimizes the sum of square errors, where an error is the difference between the actual and the predicted value of the

In [ ]:
regressor_OLS.summary()

In [ ]:
x[:3]

 ### Now we need to get rid of the variables, with the highest p-values (everything that is higher 0.05)

In [ ]:
#x5 has the highest p-value: 0.693 - this is the gender, so the gender has less  significant influence on the cost
x_opt=x[:,[0,1,2,3,4,6,7,8]]
x_opt=x_opt.astype(np.float64)
regressor_OLS=sm.OLS(endog=y, exog=x_opt).fit()
regressor_OLS.summary()

In [ ]:
x[:3]

In [ ]:
## Again x1 has the highest p-values, 0.878
x_opt=x[:,[0,2,3,4,6,7,8]]
x_opt=x_opt.astype(np.float64)
regressor_OLS=sm.OLS(endog=y,exog=x_opt).fit()
regressor_OLS.summary()

In [ ]:
x_opt=x[:,[0,2,4,6,7,8]]
x_opt=x_opt.astype(np.float64)
regressor_OLS=sm.OLS(endog=y, exog=x_opt).fit()
regressor_OLS.summary()

In [ ]:
x_opt=x[:,[0,4,6,7,8]]
x_opt=x_opt.astype(np.float64)
regressor_OLS=sm.OLS(endog=y, exog=x_opt).fit()
regressor_OLS.summary()

In [ ]:
## Now the model is ready, with the backward elimination we have left only those variables that matter 

In [ ]:
n=x[:,[0,4,6,7,8]]
n[:3]

In [ ]:
data

## Variables that are statistically significant for our prediction:
  - Constant
  - Age
  - BMI
  - Children
  - Smoker Status